<a href="https://colab.research.google.com/github/Project487/make_sprocket/blob/master/ML/Copy_of_snn_04.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
input = 1; loud = 0; E = 20

# XOR
layers = np.array([[2, 1], [2, 1], [1, 1]])
weight = np.array([3., -5., 4., -6., 2., 1.], dtype=float)
b = np.array([1., 1., -6., 5., 0.], dtype=float)
x = np.array([[0., 0.], [0., 1.], [1., 0.], [1., 1.]], dtype=float)
y = np.array([0., 1., 1., 0.], dtype=float)
lr = 5

def create_matrix(weight, b, layers, x):
    L = layers.T[0]
    k = np.cumsum(np.insert(L, 0, 0, axis=0))
    w = np.zeros((np.sum(L), np.sum(L)), dtype=float)
    wm = np.zeros((np.sum(L), np.sum(L)), dtype=float)
    bm = np.zeros(len(b), dtype=float)
    z = np.zeros((len(x), np.sum(L)), dtype=float)
    a = np.zeros((len(x), np.sum(L)), dtype=float)
    delta = np.zeros((len(x), np.sum(layers, axis=0)[0]), dtype=float)
    
    z[:,k[0]:k[1]] = x
    a[:,k[0]:k[1]] = x
    start = 0
    for i in range (0, len(L)-1):
        L_height = L[i]
        L_width = L[i+1]
        w[k[i]:k[i+1],k[i+1]:k[i+2]] = np.reshape(weight[start:start+L_width*L_height], (L_height, L_width))
        start = start+L_width*L_height 
    return k, w, wm, bm, z, a, delta

#Sigmoid activation function
def sig(z):
    return 1/(1 + np.exp(-z))

#Derivative of sigmoid
def sigd(z):
    return np.exp(z)/(1 + np.exp(z))**2

def propagate_forward(w, z, a, b, layers, x, k):
    n_inputs = len(x)
    for layer in range (1, len(layers)):
        for i in range (0, n_inputs):
            A = k[layer-1]
            B = k[layer]
            C = k[layer+1]
            z[i,B:C] = w[A:B,B:C].T @ a[i,A:B] + b[B:C]
            if layers[layer][1] == 1:
                a[i,B:C] = sig(z[i,B:C])
            if layers[layer][1] == 0:
                a[i,B:C] = z[i,B:C]
    return z, a

def calc_cost(a, y, k):
    A = a[:,k[-2]:k[-1]]
    Y = np.reshape(y, (A.shape))
    c = np.sum((A - Y)**2, axis=1)
    return c


def calc_delta(w, a, z, y, delta, k, layer, layers):    
    H = len(layers) - 1
    if layer == H:
        A = a[:,k[-2]:k[-1]]
        Y = np.reshape(y, (A.shape))        
        actv = (layers[-1,1] == 1)
        SIGD = 1
        if actv:
            SIGD = sigd(z[:,k[-2]:k[-1]])
        DELTA = 2 * (A - Y) * SIGD
        delta[:,k[-2]:k[-1]] = DELTA
    if layer < H:
        A = k[layer]
        B = k[layer+1]
        C = k[layer+2]
        DELTA = np.empty((y.shape[0],1), dtype=float)
        actv = (layers[layer,1] == 1)
        rows = y.shape[0]
        for col in range (0, B-A):
            SIGD = 1
            if actv:
                SIGD = sigd(z[:,col+A])
            d = np.reshape((np.sum(delta[:,B:C] * w[col+A,B:C], axis=1) * SIGD), (rows,1))
            DELTA = np.concatenate((DELTA, d), axis=1)
        DELTA = np.delete(DELTA, 0, axis=1)
        delta[:,A:B] = DELTA
    return delta

def calc_weight_delta(a, wm, bm, delta, k, layer):
    A = k[layer-1]
    B = k[layer]
    C = k[layer+1]
    for row in range(A, B):
        for col in range (B, C):
            wm[row][col] = np.mean(delta[:,col] * a[:,row], axis=0)      
    for col in range (B, C):
        bm[col] = np.mean(delta[:,col], axis=0)
    return wm, bm

def adjust_weights(w, wm, b, bm, lr, k, layer):
    A = k[layer-1]
    B = k[layer]
    C = k[layer+1]
    w[A:B,B:C] = w[A:B,B:C] - lr * wm[A:B,B:C]
    b[B:C] = b[B:C] - lr * bm[B:C]
    return w, b

######### Create zero matrices for future values, and fill weight matrix ##############
k, w, wm, bm, z, a, delta = create_matrix(weight, b, layers, x)

######### Training loop starts here ##############################################
for epoch in range (0, E):
    z, a = propagate_forward(w, z, a, b, layers, x, k)    
    c = calc_cost(a, y, k)
    ######### Back propagation #######################################################
    for layer in range (len(layers)-1, 0, -1):
        delta = calc_delta(w, a, z, y, delta, k, layer, layers)
        wm, bm = calc_weight_delta(a, wm, bm, delta, k, layer)
        w, b = adjust_weights(w, wm, b, bm, lr, k, layer)
    if loud:
        print("Epoch:", epoch, "Cost:", np.round(np.sum(c),3))
###### Result ########################################################################
print("Output:\n\r", np.round(a[:,k[-2]:k[-1]], 3))
print("Cost:", np.round(np.sum(c),3))

Output:
 [[0.625]
 [0.573]
 [0.598]
 [0.333]]
Cost: 0.846
